# from itertools import product / pairwise / grouped
# min/max(str1, str2)

ein target zur prüfung wählen, den rest des dicts (sample) als gegenprüfung iterieren

kürzeres(target, test) über längeres(target, test) mappen
-> in welcher position sind die meisten übereinstimmungen?
besitzen
besitzt
11111100

viele übereinstimmende positionen (vergleich mit vorigen itersteps?) -> der rest ist morphem

maps mit der größten summe = matching lexeme, oder gültige affixe wenn am wortrand

In [24]:
from src import text_utilities as tu
wordset = tu.from_path('./data/experiment/verbs')
verb_dict = tu.count_syllables(wordset)
verb_dict.keys()
for i in range(1,9): # von mächtigster menge absteigend morpheme analysieren
    print(i, len(verb_dict[i]))

1 5010
2 27261
3 30123
4 14810
5 4455
6 1005
7 193
8 32


Tokenizer-Vokabular anhand des Token-Vokabulars erstellen:

In [33]:
"""mapping 1010101 comparing words left and right"""
# durch key
maps = []
target = "verstehen"
l = len(target)
cnt = 0
cc1=0
cc2=0
cc3=0
for v in verb_dict[3]:
    pair = (v, target)
    case = tu.match_ends(v, target)
    smaller = min(pair, key=len)
    longer = max(pair, key=len)
    diff = len(longer) - len(smaller)


    if case.get("any"):
        cnt+=1
        if diff:

            if case.get("first"):
                cc1+=1
                m = []
                for i in range(len(smaller)):
                    m.append(1 if smaller[i] == longer[i] else 0)
                while len(m) < l:
                    m.append(0)
                maps.append(m)
                m = []

            if case.get("last"):
                cc2+=1
                for i in range(len(smaller)):
                    m.append(1 if smaller[i] == longer[i+diff] else 0)
                while len(m) < l:
                    m.insert(0, 0)
                maps.append(m)
                m = []

        else:
            cc3+=1
            m = []
            for i in range(l):
                    m.append(1 if pair[0][i] == pair[1][i] else 0)
            maps.append(m)
            m = []
print(cc1, cc2, cc3)

1978 2909 1630


In [34]:
# Summe aller gleichlangen maps untereinander, bildet nur Tendenz ab
#itertools recipes
new_maps = [x for x in maps if len(x)==len(target)]
[sum(x) for x in zip(*new_maps)]

[2771, 3064, 2892, 868, 692, 1179, 447, 1937, 3878]

In [35]:
# konvertiere map in string, nur vorübergehend?
str_maps = []
for m in maps:
    mstr = ""
    for i in m:
        mstr += str(i)
    str_maps.append(mstr)
print(str_maps[:10], len(str_maps))

['000000011000000001', '000000001', '000110001', '111100010', '111000010', '111100000', '000000001', '111000000', '111100000', '000000011'] 6517


In [31]:
# finde meist vorkommende patterns
# evtl längere matches gewichten?
import collections as cl
n_maps = cl.Counter(str_maps).most_common(20)
n_maps

[('000000001', 1317),
 ('111000000', 1136),
 ('000000011', 370),
 ('111100000', 277),
 ('000010001', 239),
 ('000110001', 192),
 ('111001000', 111),
 ('001000001', 89),
 ('101000000', 80),
 ('010000001', 66),
 ('111110000', 61),
 ('000000101', 56),
 ('100000000', 54),
 ('010010001', 52),
 ('111000100', 50),
 ('000100001', 47),
 ('000001011', 41),
 ('111100010', 39),
 ('000010101', 34),
 ('010110001', 30)]

In [32]:
# neue maps ignorieren unterschiede im wortinneren
import regex as re
recount_map = []
for cnt in n_maps:
    sr = re.sub(pattern='(01+0)', repl=lambda m: len(m.group(1))*"0",string=cnt[0])
    recount_map.append([sr, cnt[1]])

recount_map

[['000000001', 1317],
 ['111000000', 1136],
 ['000000011', 370],
 ['111100000', 277],
 ['000000001', 239],
 ['000000001', 192],
 ['111000000', 111],
 ['000000001', 89],
 ['100000000', 80],
 ['000000001', 66],
 ['111110000', 61],
 ['000000001', 56],
 ['100000000', 54],
 ['000000001', 52],
 ['111000000', 50],
 ['000000001', 47],
 ['000000011', 41],
 ['111100000', 39],
 ['000000101', 34],
 ['000110001', 30]]

der länge nach sortieren, siehe screenshots
str.partition() verstecken ver stecken
rpartition() -> verstecken versteck en

def predicate(shorter: str, longer: str) -> bool:
pass

#´´´
python
def predicate(shorter: str, longer: str) -> bool:
    pass


def reduce(ll: list[tuple[str, int]]):
    i = 0
    init_len = len(ll)
    # sort list by length ascending
    new_list = []
        while len(ll) > 0:
            shortest, count = ll.pop(0)
            longest = shortest
            curr_len = len(ll)
            j = 0
            while j < curr_len:
                other, other_count = ll[j]
                if predicate(shortest, other):
                ll.pop(j)
                longest = other
                count += other_count
                curr_len = len(ll)
                else:
                j+=1
                new_list.append((longest, count))

    return new_list
#´´´

In [40]:
def predicate(shorter: str, longer: str) -> bool:
    mc = shorter[0]
    nc = longer[0]
    long = max(mc, nc, key=len)
    short = min(mc, nc, key=len)
    if nc.count('1') == mc.count('1') and short in long:
        # short & long behalten
        # m, n entfernen
        # return [long, sum([m[1], n[1]])]
        return True
    else:
        return False

def index_first(map: tuple) -> str:
    """Helper function for sorted() to index the first element of a tuple in maps."""
    return map[0]

def reduce(ll: list):
    i = 0
    # init_len = len(ll)
    # sort list by length ascending
    new_list = []
    while len(ll) > 0:
        shortest, count = ll.pop(0)
        longest = shortest
        curr_len = len(ll)
        j = 0
        while j < curr_len:
            other, other_count = ll[j]
            if predicate(shortest, other):
                ll.pop(j)
                longest = other
                count += other_count
                curr_len = len(ll)
            else:
                j+=1
                new_list.append((longest, count))

    return new_list

In [45]:

sorted_maps = sorted(recount_map, key=lambda t: t[0])
sorted_maps

In [42]:
def combine_counts(m: list, n: list):
    """Compares two maps e.g.['1100000', 46] vs. ['1000000', 29]. Sums count if (m in n) or vice versa and has equal number of 1s. Map must be str(int), count must be int."""
    mc = m[0]
    nc = n[0]
    long = max(mc, nc, key=len)
    short = min(mc, nc, key=len)
    if nc.count('1') == mc.count('1') and short in long:
        # short & long behalten
        # m, n entfernen
        return [long, sum([m[1], n[1]])]
    else:
        return None

In [43]:
import itertools as it
for m, n in set(it.combinations(recount_map, 2)):
    print(m, n)

#def pairwise(iterable):
#    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
#    a = iter(iterable)
#    return zip(a, a)

left = [x for x in recount_map if x[0].startswith("1") and x[0].endswith("0")]
right = [x for x in recount_map if x[0].startswith("0") and x[0].endswith("1")]
circ = [x for x in recount_map if x[0].startswith("1") and x[0].endswith("1")]

print(left, right, circ)

TypeError: unhashable type: 'list'

In [44]:
def map_subword(target: str, map:str) -> str:
    """Returns a subword from a target string and a map. Yet to implement maps with 1 on both ends."""
    if map.startswith("1"):
        return target[:map.count("1")]+"##"
    elif map.endswith("1"):
        return "##"+target[-map.count("1"):]
print(map_subword("verstehen", "000000011"))

##en
